# Python 2 HSUTCC: Session 8: Decorator


In [1]:
from collections.abc import Callable
import functools

## What is a decorator?


Decorator is a function that recieve a function and then return something. For example,


In [2]:
def my_decorator(target_func: Callable) -> int:
    return 20

In [ ]:
def foo() -> None:
    print('I am silly.')


bar = my_decorator(foo)
print(bar)

20


Now, to use a decorator, we have a shorthand or as people normally call **Syntactic Sugar** using `@` sign as follow.


In [ ]:
@my_decorator
def foo() -> None:
    print('I am silly.')


foo()

TypeError: 'int' object is not callable

In [5]:
print(foo)

20


What happend? How come our `foo` function become an integer? Well, the thing that happend behind the hood is


In [ ]:
# function object -> <1234>
# foo -> <1234>
def foo() -> None:
    print('I am silly.')


# foo = 20
foo = my_decorator(foo)

print(foo)

20


In [ ]:
@my_decorator
def foo() -> None:
    print(2 + 4)
    print('I am silly.')


print(foo)

20


## Heck! What are we talking about?


### Adding functionalities, the naked way.


Let's take a step back and imagine some scenario.


Say you are working on a project where you have a send notification function implemented where, by default you send the notification via email to a list of contact.


In [8]:
contact = ['Ve', 'Due', 'Batman', 'Superman']

In [9]:
def send_noti(contact: list) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

In [10]:
send_noti(contact)

Send email to Ve.
Send email to Due.
Send email to Batman.
Send email to Superman.


Now, what happend if you want to send the notification via SMS as well?


In [ ]:
def send_noti(contact: list, via_sms: bool = False) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

    if via_sms:
        for each_contact in contact:
            print(f'Send SMS to {each_contact}.')

In [12]:
send_noti(contact, via_sms=True)

Send email to Ve.
Send email to Due.
Send email to Batman.
Send email to Superman.
Send SMS to Ve.
Send SMS to Due.
Send SMS to Batman.
Send SMS to Superman.


Now, if we add more channels, our code will become something like


In [ ]:
def send_noti(
    contact: list,
    via_sms: bool = False,
    via_fb: bool = False,
    via_slack: bool = False
) -> None:

    for each_contact in contact:
        print(f'Send email to {each_contact}.')

    if via_sms:
        for each_contact in contact:
            print(f'Send SMS to {each_contact}.')

    if via_fb:
        for each_contact in contact:
            print(f'Send SMS to {each_contact}.')

    if via_slack:
        for each_contact in contact:
            print(f'Send SMS to {each_contact}.')

This is one way that we can add more functionalities to our function (we start with a function that send notification via email and then we add SMS, Facebook, and Slack to the functionality). With this concept of adding functionality on top of each other, we have another method - using decorator.


### Adding functionalities, the decorated way.


Let's take a look at the normal structure of decorator (wrapper).


In [14]:
def wrapper(my_func):

    def inner_func(*args, **kwargs):
        # Do something useful
        pass

    return inner_func

Again, begining with our base `send_noti` function.


In [15]:
def send_noti(contact: list) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

To add the SMS functionality, we will create a new decorator.


In [20]:
def sms_decorator(func: Callable[[list], None]) -> None:
    def inner(contact):
        for each_contact in contact:
            print(f'Send SMS to {each_contact}.')

        func(contact)

    return inner

In [21]:
# @sms_decorator
# def send_noti(contact: list) -> None: # <123>
#     for each_contact in contact:
#         print(f'Send email to {each_contact}.')

# send_noti = sms_decorator(send_noti)

# def sms_decorator(func): # <123>
#     def inner(contact): # <124>
#         for each_contact in contact:
#             print(f'Send SMS to {each_contact}.')

#         func(contact) # <123>

#     return inner # <124>

# send_noti # <124>

In [ ]:
@sms_decorator
def send_noti(contact: list) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')


send_noti(contact)

Send SMS to Ve.
Send SMS to Due.
Send SMS to Batman.
Send SMS to Superman.
Send email to Ve.
Send email to Due.
Send email to Batman.
Send email to Superman.


Voilà, seem like our code work correctly. Let's stop for a moment and disect what going on.


```python
def send_noti(contact: list) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')
```


```python
send_noti = sms_decorator(send_noti)
```


What happend here?


```python
def inner(contact):
    for each_contact in contact:
        print(f'Send SMS to {each_contact}.')

    send_noti(contact)
```


```python
send_noti = inner
```


Now, the `send_noti` becomes


```python
def send_noti(contact):
    for each_contact in contact:
        print(f'Send SMS to {each_contact}.')

    # the base send_noti
    send_noti(contact)
```


```python
send_noti(contact)
```


Okayyyyy, take a deep breathe and process what just happend. We always have time.


Ready? Now, to add more functionalities, we can do the same.


In [ ]:
def fb_decorator(func: Callable[[list], None]) -> None:
    def inner(contact):
        for each_contact in contact:
            print(f'Send FB to {each_contact}.')

        func(contact)

    return inner


def slack_decorator(func: Callable[[list], None]) -> None:
    def inner(contact):
        for each_contact in contact:
            print(f'Send Slack to {each_contact}.')

        func(contact)

    return inner

In [ ]:
@sms_decorator
@fb_decorator
@slack_decorator
def send_noti(contact: list) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')


send_noti(contact)

Send SMS to Ve.
Send SMS to Due.
Send SMS to Batman.
Send SMS to Superman.
Send FB to Ve.
Send FB to Due.
Send FB to Batman.
Send FB to Superman.
Send Slack to Ve.
Send Slack to Due.
Send Slack to Batman.
Send Slack to Superman.
Send email to Ve.
Send email to Due.
Send email to Batman.
Send email to Superman.


#### Sidenote: Function Documentation


Suppose that we have a documentation for our `send_noti` function.


In [25]:
def send_noti(contact: list) -> None:
    """Send notification to contacts"""
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

If you check the documentation by hovering over the function or using the magic command `?`, you will see the documentation.


In [26]:
?send_noti

Signature: send_noti(contact: list) -> None
Docstring: Send notification to contacts
File:      /var/folders/p0/j7kbnk513g96xctv2zdsyks80000gn/T/ipykernel_5507/135047537.py
Type:      function

In [27]:
help(send_noti)

Help on function send_noti in module __main__:

send_noti(contact: list) -> None
    Send notification to contacts



Now, let's add the decorator and see what happend.


In [31]:
def sms_decorator(func: Callable[[list], None]) -> None:
    def inner(contact):
        # """This documentation is from the decorator!"""
        for each_contact in contact:
            print(f'Send SMS to {each_contact}.')

        func(contact)

    return inner

In [32]:
@sms_decorator
def send_noti(contact: list) -> None:
    """Send notification to contacts"""
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

In [33]:
?send_noti

Signature: send_noti(contact)
Docstring: <no docstring>
File:      /var/folders/p0/j7kbnk513g96xctv2zdsyks80000gn/T/ipykernel_5507/4066696104.py
Type:      function

The documentation is gone! This is not desirable at all. To fix this, we will need a `functools` module.


In [ ]:
def sms_decorator(func: Callable[[list], None]) -> None:

    @functools.wraps(func)
    def inner():
        for each_contact in contact:
            print(f'Send SMS to {each_contact}.')

        func(contact)

    return inner

In [35]:
@sms_decorator
def send_noti(contact: list) -> None:
    """Send notification to contacts"""
    for each_contact in contact:
        print(f'Send email to {each_contact}.')

In [36]:
?send_noti

Signature: send_noti(contact: list) -> None
Docstring: Send notification to contacts
File:      /var/folders/p0/j7kbnk513g96xctv2zdsyks80000gn/T/ipykernel_5507/1929091552.py
Type:      function

All is good now.


## Some examples


### @timethis


In [ ]:
import time


def timethis(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, end=" ... ")

        start_time = time.time()
        result = func(*args, **kwargs)
        elapse_time = time.time() - start_time

        print(elapse_time)
        return result
    return inner

In [ ]:
@timethis
def add(a, b):
    return a + b


add(2, 5)

add ... 0.0


7

In [ ]:
@timethis
def factorial(number):
    result = 1

    for current_num in range(1, number + 1):
        result *= current_num

    return result


factorial(1000)

factorial ... 0.0003292560577392578


4023872600770937735437024339230039857193748642107146325437999104299385123986290205920442084869694048004799886101971960586316668729948085589013238296699445909974245040870737599188236277271887325197795059509952761208749754624970436014182780946464962910563938874378864873371191810458257836478499770124766328898359557354325131853239584630755574091142624174743493475534286465766116677973966688202912073791438537195882498081268678383745597317461360853795345242215865932019280908782973084313928444032812315586110369768013573042161687476096758713483120254785893207671691324484262361314125087802080002616831510273418279777047846358681701643650241536913982812648102130927612448963599287051149649754199093422215668325720808213331861168115536158365469840467089756029009505376164758477284218896796462449451607653534081989013854424879849599533191017233555566021394503997362807501378376153071277619268490343526252000158885351473316117021039681759215109077880193931781141945452572238655414610628921879602238389714760

In [ ]:
import time

# A bigger version


def timethis(func=None, *, n_iter=100):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, end=" ... ")

        acc = float("inf")
        for _ in range(n_iter):
            tick = time.perf_counter()
            result = func(*args, **kwargs)
            acc = min(acc, time.perf_counter() - tick)
        print(acc)

        return result
    return inner


result = timethis(sum)(range(10 ** 6))
print(result)

### @once


In [41]:
def once(func):

    @functools.wraps(func)
    def inner(*args, **kwargs):

        if not inner.called:
            func(*args, **kwargs)
            inner.called = True

    inner.called = False

    return inner

In [ ]:
def once_v2(func):

    called = False

    @functools.wraps(func)
    def inner(*args, **kwargs):
        nonlocal called

        if not called:
            func(*args, **kwargs)
            called = True

    print(locals())

    return inner

In [ ]:
@once_v2
def print_3():
    print(3)


for _ in range(1000):
    print_3()

{'func': <function print_3 at 0x10dc9cb80>, 'inner': <function print_3 at 0x10dc9ccc0>, 'called': False}
3


In [52]:
print(globals())

{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'from collections.abc import Callable\nimport functools', 'def my_decorator(target_func: Callable) -> int:\n    return 20', "def foo() -> None:\n    print('I am silly.')\n\nbar = my_decorator(foo)\nprint(bar)", "@my_decorator\ndef foo() -> None:\n    print('I am silly.')\n\nfoo()", 'print(foo)', "# function object -> <1234>\n# foo -> <1234>\ndef foo() -> None:\n    print('I am silly.')\n\n# foo = 20\nfoo = my_decorator(foo)\n\nprint(foo)", "@my_decorator\ndef foo() -> None:\n    print('I am silly.')\n\nprint(foo)", "contact = ['Ve', 'Due', 'Batman', 'Superman']", "def send_noti(contact: list) -> None:\n    for each_contact in contact:\n        print(f'Send email to {each_contact}.')", 'send_noti(contact)', "def send_noti

### @memoized


In [ ]:
def memoized(func):
    cache = {}

    @functools.wraps(func)
    def inner(*args, **kwargs):
        key = args + tuple(sorted(kwargs.items()))
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]

    return inner


@memoized
def ackermann(m, n):
    if not m:
        return n + 1
    elif not n:
        return ackermann(m - 1, 1)
    else:
        return ackermann(m - 1, ackermann(m, n - 1))


ackermann(3, 4)
ackermann(3, 4)

## Decorator with arguments


What if we want to add some arguments for the decorator? For example, initializing the origin address for SMS messaging.


In [ ]:
def sms_decorator(func: Callable[[list], None], origin: str) -> None:

    @functools.wraps(func)
    def inner(contact):
        for each_contact in contact:
            print(f'Send SMS from {origin} to {each_contact}.')

        func(contact)

    return inner


@sms_decorator('Salmon3748')
def send_noti(contact: list) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')


send_noti(contact)

TypeError: sms_decorator() missing 1 required positional argument: 'origin'

If we do that directly in the decorator definition, we wouldn't be able to go to sleep (since the code will break). In order to do that, we will introduce a new layer of decorator - `with_arguments`.


In [57]:
def with_arguments(deco):
    @functools.wraps(deco)
    def wrapper(*dargs, **dkwargs):

        @functools.wraps(wrapper)
        def decorator(func):
            result = deco(func, *dargs, **dkwargs)
            return result

        return decorator
    return wrapper

Let's see it in action.


In [ ]:
# sms_decorator = with_arguments(sms_decorator) # <123>

# def with_arguments(deco):
#     @functools.wraps(deco)
#     def wrapper(*dargs, **dkwargs): # <124>

#         @functools.wraps(wrapper)
#         def decorator(func): # <125>
#             result = deco(func, *dargs, **dkwargs)
#             return result

#         return decorator
#     return wrapper

# # sms_decorator <124>

# @sms_decorator('Salmon3748')
# def send_noti(contact: list) -> None:
#     for each_contact in contact:
#         print(f'Send email to {each_contact}.')

# sms_decorator = sms_decorator('Salmon3748') # <124>
# # sms_decorator <125>
# # send_noti <126>

# send_noti = sms_decorator(send_noti)
# # deco(func, *dargs, **dkwargs) <123>
# # <123>(<126>, 'Salmon3748')
# def sms_decorator(func: Callable[[list], None], origin: str) -> None:

#     @functools.wraps(func)
#     def inner(contact):
#         for each_contact in contact:
#             print(f'Send SMS from {origin} to {each_contact}.')

#         func(contact)

#     return inner

In [ ]:
@with_arguments
def sms_decorator(func: Callable[[list], None], origin: str) -> None:

    @functools.wraps(func)
    def inner(contact):
        for each_contact in contact:
            print(f'Send SMS from {origin} to {each_contact}.')

        func(contact)

    return inner


@sms_decorator('Salmon3748')
def send_noti(contact: list) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')


send_noti(contact)

Send SMS from Salmon3748 to Ve.
Send SMS from Salmon3748 to Due.
Send SMS from Salmon3748 to Batman.
Send SMS from Salmon3748 to Superman.
Send email to Ve.
Send email to Due.
Send email to Batman.
Send email to Superman.


### Sidenote: Code Breakdown


```python
@with_arguments
def sms_decorator(func, origin):

    @functools.wraps(func)
    def inner(contact):
        for each_contact in contact:
            print(f'Send SMS from {origin} to {each_contact}.')

        func(contact)

    return inner
```


The normal decorator call


```python
sms_decorator = with_arguments(sms_decorator)
```


The function wrapper is created.


```python
def wrapper(*dargs, **dkwargs):

    @functools.wraps(wrapper)
    def decorator(func):
        result = sms_decorator(func, *dargs, **dkwargs)
        return result

    return decorator

sms_decorator = wrapper
```


Now, the decorator is used.


```python
@sms_decorator('Salmon3748')
def send_noti(contact):
    for each_contact in contact:
        print(f'Send email to {each_contact}.')
```


Initializing a decorator with arguments is a two-step process, first, the arguments are passed into the `wrapper` function and `decorator` function is defined.


```python
deco = sms_decorator('Salmon3748')

def decorator(func):
    result = sms_decorator(func, 'Salmon3748')
    return result
```


Then the decorator is passed in the function as usual.


```python
send_noti = deco(send_noti)

result = sms_decorator(send_noti, 'Salmon3748')
```


And the process continue...


## Decorators with optional arguments


To create an optional argument is now easy enough.


In [ ]:
@with_arguments
def sms_decorator(func: Callable[[list], None], origin: str = 'Salmon3748') -> None:

    @functools.wraps(func)
    def inner(contact):
        for each_contact in contact:
            print(f'Send SMS from {origin} to {each_contact}.')

        func(contact)

    return inner


@sms_decorator()
def send_noti(contact: list) -> None:
    for each_contact in contact:
        print(f'Send email to {each_contact}.')


send_noti(contact)

Send SMS from Salmon3748 to Ve.
Send SMS from Salmon3748 to Due.
Send SMS from Salmon3748 to Batman.
Send SMS from Salmon3748 to Superman.
Send email to Ve.
Send email to Due.
Send email to Batman.
Send email to Superman.


In [ ]:
# Function Namespace
# Module Namespace
# Class Namespace

name = 'Luna'


class Cat:
    name = 'Oscar'


cat = Cat()
cat.name

'Oscar'

# Tasks (18 November 2025)


1. Write a decorator `n_times` which given a number `n` makes a function to execute `n` times.


```python
@n_times(3)
def print_my_name():
    print('Due')

print_my_name()
```


```terminal
Due
Due
Due
```


In [ ]:
from collections.abc import Callable


def with_arguments(deco: Callable):
    def wrapper(*dargs, **dkwargs):
        def decorator(func):
            result = deco(func, *dargs, **dkwargs)
            return result
        return decorator
    return wrapper


@with_arguments
def n_times(func: Callable, n: int):
    def inner(*args, **kwargs):
        for _ in range(n):
            func(*args, **kwargs)
    return inner


@n_times(3)
def print_my_name():
    print('Yan')


print_my_name()

Yan
Yan
Yan


2. Write a decorator `once` which makes a function to execute only once and other times it just returns the result calculated on the first call.


```python
@once
def random_number():
    return random.randint(1, 10)

print(random_number())
print(random_number())
print(random_number())
```


In [ ]:
import random


def once(func):
    result = None

    def inner(*args, **kwargs):
        nonlocal result
        if result is None:
            result = func(*args, **kwargs)
        return result
    return inner


@once
def random_number():
    return random.randint(1, 10)


print(random_number())
print(random_number())
print(random_number())

"""
Because I submit this one 2 days earlier, please forgive me that I was later around 30 minutes for previous one. please
"""

7
7
7
